In [1]:
#---Data---#
#Tasks
task = [:t1, :t2, :t3, :t4, :t5, :t6, :t7, :t8, :t9,            
        :t10, :t11, :t12, :t13, :t14, :t15, :t16, :t17, :t18]

#Project time spans
sp = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1]
span = Dict(zip(task,sp))

#Task predecessors
pr = ( [], [:t1], [:t2], [:t2], [:t3], [:t4, :t5], [:t4], [:t6], [:t4, :t6], [:t4], [:t6], 
       [:t9],[:t7], [:t2], [:t4, :t14], [:t8, :t11, :t14], [:t12], [:t17])
pred = Dict(zip(task,pr))

#Maximum task week reduction 
mR = [0 3 1 2 2 1 1 0 2 1 1 0 0 2 2 1 3 0]
maxRed = Dict(zip(task, mR))

#Reduction cost per week
rC = [0 30 26 12 17 15 8 0 42 21 18 0 0 22 12 6 16 0]
redCost = Dict(zip(task, rC))


#---Model without task reduction---#
using JuMP
m = Model()

#Start time of tasks

@defVar(m, startTime[task] >= 0 )                
#Predecessors must be completed before each task
@addConstraint(m, completePred[t in task,p in pred[t]], startTime[t] >= startTime[p] + span[p])

#Minimimize completion time of final task
@setObjective(m, Min, startTime[task[end]] + span[task[end]])

#Display completion time
solve(m)
println("Original completion time: ",getObjectiveValue(m), " weeks")  

#---Model with task reduction----#
m2 = Model()

incentive = 30000                         #Incentive per week early

@defVar(m2, startTime[task] >= 0 )        #Start time of tasks
#Actual span of tasks with reductions
@defVar(m2, actualSpan[task] >= 0)

#Total incentive received
@defExpr(totalIncentive, (getObjectiveValue(m) - (startTime[task[end]] + actualSpan[task[end]]))*incentive)
#Total reduction cost
@defExpr(totalRedCost, sum{(span[t] - actualSpan[t])*redCost[t], t in task})

#Predecessors must be completed before each task
@addConstraint(m2, completePred[t in task,p in pred[t]], startTime[t] >= startTime[p] + actualSpan[p])
#Constraints on task reduction
@addConstraint(m2, redCon[t in task], span[t] - actualSpan[t] <= maxRed[t])
@addConstraint(m2, redCon2[t in task], span[t] >= actualSpan[t])

@setObjective(m2, Max, totalIncentive - totalRedCost)    #Maximize profit

solve(m2)
#Display completion time
println("Completion time with task reduction: ", 
getValue(startTime[task[end]]) + getValue(actualSpan[task[end]]), " weeks")

Original completion time: 64.0 weeks
Completion time with task reduction: 52.0 weeks
